In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df = pd.read_csv('P_TD_LTV_extrapolation.csv')

In [3]:
df.isna().sum()

,0
customer_name,0
subscription_date,0
maturity,0
subscription_bom,0


In [4]:
"""
Cohort size here is the number of customers from a cohort
that reached maturity in a column
"""
group = df.groupby(['subscription_bom', 'maturity'])
cohort_data = group['customer_name'].nunique().reset_index().rename(columns = {'customer_name':'cohort_size'})

cohort_data

,subscription_bom,maturity,cohort_size
0,2019-12-01,0,9056
1,2019-12-01,1,3805
2,2019-12-01,2,2575
3,2019-12-01,3,1835
4,2019-12-01,4,1410
...,...,...,...
2341,2025-05-01,1,521
2342,2025-05-01,2,543
2343,2025-06-01,0,1708
2344,2025-06-01,1,859


In [5]:
"""
this matrix has 2 axes:
x - cohort date (beginning of month); y - cohort maturity (in months)
every cell is a unique count of customer for the x and y

"""
retention = cohort_data.pivot(index='subscription_bom', columns='maturity', values='cohort_size').sort_index(ascending = True)

retention

maturity,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,9056.0,3805.0,2575.0,1835.0,1410.0,1135.0,986.0,657.0,327.0,421.0,...,31.0,28.0,32.0,29.0,28.0,25.0,18.0,28.0,22.0,24.0
2020-01-01,12376.0,5349.0,3276.0,2554.0,1911.0,1589.0,1034.0,977.0,688.0,892.0,...,27.0,30.0,27.0,21.0,21.0,22.0,23.0,17.0,17.0,NaN
2020-02-01,12368.0,5144.0,3219.0,2603.0,1817.0,1228.0,1184.0,745.0,997.0,827.0,...,22.0,28.0,21.0,20.0,17.0,28.0,22.0,23.0,NaN,NaN
2020-03-01,12316.0,5510.0,3453.0,2225.0,1691.0,1448.0,951.0,1160.0,980.0,727.0,...,25.0,29.0,27.0,15.0,29.0,18.0,18.0,NaN,NaN,NaN
2020-04-01,25561.0,11128.0,7950.0,4366.0,3791.0,2415.0,2769.0,2309.0,1742.0,1441.0,...,56.0,40.0,46.0,63.0,45.0,52.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,2328.0,822.0,348.0,342.0,313.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-01,1666.0,631.0,351.0,374.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-01,1639.0,521.0,543.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
retention_sub = retention

In [7]:
cohorts = retention_sub[0].reset_index().rename(columns={0:'cohort_size'})
cohorts

,subscription_bom,cohort_size
0,2019-12-01,9056.0
1,2020-01-01,12376.0
2,2020-02-01,12368.0
3,2020-03-01,12316.0
4,2020-04-01,25561.0
...,...,...
63,2025-03-01,2328.0
64,2025-04-01,1666.0
65,2025-05-01,1639.0
66,2025-06-01,1708.0


In [8]:
"""
Cohort retention matrix shows how many of the cohort
moved from maturity t to maturity t+1 (renewed sunscription)
"""
retention_matrix = retention_sub.divide(retention_sub[0], axis=0).round(4)
retention_matrix.sort_index
retention_matrix

maturity,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,1.0,0.4202,0.2843,0.2026,0.1557,0.1253,0.1089,0.0725,0.0361,0.0465,...,0.0034,0.0031,0.0035,0.0032,0.0031,0.0028,0.0020,0.0031,0.0024,0.0027
2020-01-01,1.0,0.4322,0.2647,0.2064,0.1544,0.1284,0.0835,0.0789,0.0556,0.0721,...,0.0022,0.0024,0.0022,0.0017,0.0017,0.0018,0.0019,0.0014,0.0014,NaN
2020-02-01,1.0,0.4159,0.2603,0.2105,0.1469,0.0993,0.0957,0.0602,0.0806,0.0669,...,0.0018,0.0023,0.0017,0.0016,0.0014,0.0023,0.0018,0.0019,NaN,NaN
2020-03-01,1.0,0.4474,0.2804,0.1807,0.1373,0.1176,0.0772,0.0942,0.0796,0.0590,...,0.0020,0.0024,0.0022,0.0012,0.0024,0.0015,0.0015,NaN,NaN,NaN
2020-04-01,1.0,0.4354,0.3110,0.1708,0.1483,0.0945,0.1083,0.0903,0.0682,0.0564,...,0.0022,0.0016,0.0018,0.0025,0.0018,0.0020,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,1.0,0.3531,0.1495,0.1469,0.1345,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-01,1.0,0.3788,0.2107,0.2245,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-01,1.0,0.3179,0.3313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
retention_matrix.shape[1]

68

In [10]:
df_zero = pd.DataFrame(np.zeros((retention_matrix.shape[0]
                       , retention_matrix.shape[1]))
                       , index = retention_sub.index)
df_zero

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
"""
Marginal retention matrix is calculated for all the columns except first
by division of a column that corresponds to t+1 maturity by t
In other words, it tells us how many customers who reached maturity t moved to t+1
"""

df_marginal = df_zero.copy()

# dividing each column (except ind = 0) by the previous one
#for i in np.unique(df.order_age_wmax.values):
for i in retention_matrix.columns.values:
  if i == 0:
    df_marginal.iloc[:, i] = retention_sub[i].divide(retention_sub[i], axis = 0)
  else:
    df_marginal.iloc[:, i] = retention_sub[i].divide(retention_sub[i-1], axis = 0)

df_marginal

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,1.0,0.420163,0.676741,0.712621,0.768392,0.804965,0.868722,0.666329,0.497717,1.287462,...,0.885714,0.903226,1.142857,0.906250,0.965517,0.892857,0.720000,1.555556,0.785714,1.090909
2020-01-01,1.0,0.432207,0.612451,0.779609,0.748238,0.831502,0.650724,0.944874,0.704197,1.296512,...,1.038462,1.111111,0.900000,0.777778,1.000000,1.047619,1.045455,0.739130,1.000000,NaN
2020-02-01,1.0,0.415912,0.625778,0.808636,0.698041,0.675839,0.964169,0.629223,1.338255,0.829488,...,0.846154,1.272727,0.750000,0.952381,0.850000,1.647059,0.785714,1.045455,NaN,NaN
2020-03-01,1.0,0.447386,0.626679,0.644367,0.760000,0.856298,0.656768,1.219769,0.844828,0.741837,...,0.781250,1.160000,0.931034,0.555556,1.933333,0.620690,1.000000,NaN,NaN,NaN
2020-04-01,1.0,0.435351,0.714414,0.549182,0.868301,0.637035,1.146584,0.833875,0.754439,0.827210,...,0.933333,0.714286,1.150000,1.369565,0.714286,1.155556,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,1.0,0.353093,0.423358,0.982759,0.915205,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-01,1.0,0.378752,0.556260,1.065527,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-01,1.0,0.317877,1.042226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
"""
Marginal retention by taking an average of the previous N rows
Here is where you would like to consider seasonality.

Notice that using mean() function can be not always desirable.
This is due to a reason that some values in previous rows might contain 0's.
We want to average non-zero values in the previous N rows.
Including zeros will significantly affect the resulting retention matrix.
"""
N = 3 # periods to be averaged
df_marginal_e = df_marginal.copy()
for row_n in range(df_marginal.shape[0])[1:]:
    row = df_marginal_e.iloc[row_n, :]
    mask = (row.isna()) | (row == 0) #missing values to be filled
    indx = np.where(mask)[0] #indices of missing values in a row
    if row_n <= N:
      df_marginal_e.iloc[row_n, indx] = np.sum(df_marginal_e.iloc[:row_n, indx], axis = 0) / np.count_nonzero(df_marginal_e.iloc[:row_n, indx], axis = 0)
    else:
      # print(row_n)
      df_marginal_e.iloc[row_n, indx] = np.sum(df_marginal_e.iloc[row_n - N:row_n, indx], axis = 0) / np.count_nonzero(df_marginal_e.iloc[row_n - N:row_n, indx], axis = 0)
      #print(df_marginal_e.iloc[row_n, indx])
df_marginal_e

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,1.0,0.420163,0.676741,0.712621,0.768392,0.804965,0.868722,0.666329,0.497717,1.287462,...,0.885714,0.903226,1.142857,0.906250,0.965517,0.892857,0.720000,1.555556,0.785714,1.090909
2020-01-01,1.0,0.432207,0.612451,0.779609,0.748238,0.831502,0.650724,0.944874,0.704197,1.296512,...,1.038462,1.111111,0.900000,0.777778,1.000000,1.047619,1.045455,0.739130,1.000000,1.090909
2020-02-01,1.0,0.415912,0.625778,0.808636,0.698041,0.675839,0.964169,0.629223,1.338255,0.829488,...,0.846154,1.272727,0.750000,0.952381,0.850000,1.647059,0.785714,1.045455,0.892857,1.090909
2020-03-01,1.0,0.447386,0.626679,0.644367,0.760000,0.856298,0.656768,1.219769,0.844828,0.741837,...,0.781250,1.160000,0.931034,0.555556,1.933333,0.620690,1.000000,1.113380,0.892857,1.090909
2020-04-01,1.0,0.435351,0.714414,0.549182,0.868301,0.637035,1.146584,0.833875,0.754439,0.827210,...,0.933333,0.714286,1.150000,1.369565,0.714286,1.155556,0.943723,0.965988,0.928571,1.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,1.0,0.353093,0.423358,0.982759,0.915205,0.791933,0.878576,0.837387,0.916646,0.903162,...,1.008092,0.987194,0.907249,1.048161,1.042460,1.059184,0.936147,1.028363,0.910714,1.090909
2025-04-01,1.0,0.378752,0.556260,1.065527,0.838320,0.755364,0.954355,0.810255,0.917526,0.916284,...,1.008092,0.987194,0.907249,1.048161,1.042460,1.059184,0.936147,1.028363,0.910714,1.090909
2025-05-01,1.0,0.317877,1.042226,0.963516,0.806534,0.820806,0.922088,0.801066,0.930190,0.912765,...,1.008092,0.987194,0.907249,1.048161,1.042460,1.059184,0.936147,1.028363,0.910714,1.090909


In [13]:
retention_matrix_e = retention_matrix.copy()
columns = retention_matrix_e.columns.values #assuming those are integers ????? - rethink
"""
Here we extrapolate Cohort Retention Matrix
We do this by i'th columns in marginal retention matrix by (i-1)th column in cohort retention matrix.
We only do it for 0 and NA values
"""
for column in columns[1:]:
  #row_indx = np.where(retention_matrix_e.loc[:, column].isna())[0] #OR 0??
  row_indx = np.where((retention_matrix_e.loc[:, column].isna()) | (retention_matrix_e.loc[:, column] == 0))[0] #OR 0??
  retention_matrix_e.iloc[row_indx, column] = retention_matrix_e.iloc[row_indx, column - 1] * df_marginal_e.iloc[row_indx, column]

retention_matrix_e

maturity,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,1.0,0.420200,0.284300,0.202600,0.155700,0.125300,0.108900,0.072500,0.036100,0.046500,...,0.003400,0.003100,0.003500,0.003200,0.003100,0.002800,0.002000,0.003100,0.002400,0.002700
2020-01-01,1.0,0.432200,0.264700,0.206400,0.154400,0.128400,0.083500,0.078900,0.055600,0.072100,...,0.002200,0.002400,0.002200,0.001700,0.001700,0.001800,0.001900,0.001400,0.001400,0.001527
2020-02-01,1.0,0.415900,0.260300,0.210500,0.146900,0.099300,0.095700,0.060200,0.080600,0.066900,...,0.001800,0.002300,0.001700,0.001600,0.001400,0.002300,0.001800,0.001900,0.001696,0.001851
2020-03-01,1.0,0.447400,0.280400,0.180700,0.137300,0.117600,0.077200,0.094200,0.079600,0.059000,...,0.002000,0.002400,0.002200,0.001200,0.002400,0.001500,0.001500,0.001670,0.001491,0.001627
2020-04-01,1.0,0.435400,0.311000,0.170800,0.148300,0.094500,0.108300,0.090300,0.068200,0.056400,...,0.002200,0.001600,0.001800,0.002500,0.001800,0.002000,0.001887,0.001823,0.001693,0.001847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,1.0,0.353100,0.149500,0.146900,0.134500,0.106515,0.093582,0.078364,0.071832,0.064876,...,0.025077,0.024756,0.022460,0.023542,0.024541,0.025994,0.024334,0.025024,0.022790,0.024862
2025-04-01,1.0,0.378800,0.210700,0.224500,0.188203,0.142162,0.135673,0.109930,0.100863,0.092419,...,0.035626,0.035170,0.031908,0.033444,0.034864,0.036928,0.034570,0.035550,0.032376,0.035320
2025-05-01,1.0,0.317900,0.331300,0.319213,0.257456,0.211321,0.194857,0.156093,0.145196,0.132530,...,0.051071,0.050417,0.045741,0.047944,0.049980,0.052938,0.049557,0.050963,0.046413,0.050632


In [14]:
retention_matrix_e2 = retention_matrix_e.multiply(retention_sub[0], axis=0).round(0)
retention_matrix_e2

maturity,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,9056.0,3805.0,2575.0,1835.0,1410.0,1135.0,986.0,657.0,327.0,421.0,...,31.0,28.0,32.0,29.0,28.0,25.0,18.0,28.0,22.0,24.0
2020-01-01,12376.0,5349.0,3276.0,2554.0,1911.0,1589.0,1033.0,976.0,688.0,892.0,...,27.0,30.0,27.0,21.0,21.0,22.0,24.0,17.0,17.0,19.0
2020-02-01,12368.0,5144.0,3219.0,2603.0,1817.0,1228.0,1184.0,745.0,997.0,827.0,...,22.0,28.0,21.0,20.0,17.0,28.0,22.0,23.0,21.0,23.0
2020-03-01,12316.0,5510.0,3453.0,2226.0,1691.0,1448.0,951.0,1160.0,980.0,727.0,...,25.0,30.0,27.0,15.0,30.0,18.0,18.0,21.0,18.0,20.0
2020-04-01,25561.0,11129.0,7949.0,4366.0,3791.0,2416.0,2768.0,2308.0,1743.0,1442.0,...,56.0,41.0,46.0,64.0,46.0,51.0,48.0,47.0,43.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,2328.0,822.0,348.0,342.0,313.0,248.0,218.0,182.0,167.0,151.0,...,58.0,58.0,52.0,55.0,57.0,61.0,57.0,58.0,53.0,58.0
2025-04-01,1666.0,631.0,351.0,374.0,314.0,237.0,226.0,183.0,168.0,154.0,...,59.0,59.0,53.0,56.0,58.0,62.0,58.0,59.0,54.0,59.0
2025-05-01,1639.0,521.0,543.0,523.0,422.0,346.0,319.0,256.0,238.0,217.0,...,84.0,83.0,75.0,79.0,82.0,87.0,81.0,84.0,76.0,83.0


In [15]:
flipped_df = retention_matrix_e2.iloc[:, ::-1]

flipped_df

maturity,67,66,65,64,63,62,61,60,59,58,...,9,8,7,6,5,4,3,2,1,0
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,24.0,22.0,28.0,18.0,25.0,28.0,29.0,32.0,28.0,31.0,...,421.0,327.0,657.0,986.0,1135.0,1410.0,1835.0,2575.0,3805.0,9056.0
2020-01-01,19.0,17.0,17.0,24.0,22.0,21.0,21.0,27.0,30.0,27.0,...,892.0,688.0,976.0,1033.0,1589.0,1911.0,2554.0,3276.0,5349.0,12376.0
2020-02-01,23.0,21.0,23.0,22.0,28.0,17.0,20.0,21.0,28.0,22.0,...,827.0,997.0,745.0,1184.0,1228.0,1817.0,2603.0,3219.0,5144.0,12368.0
2020-03-01,20.0,18.0,21.0,18.0,18.0,30.0,15.0,27.0,30.0,25.0,...,727.0,980.0,1160.0,951.0,1448.0,1691.0,2226.0,3453.0,5510.0,12316.0
2020-04-01,47.0,43.0,47.0,48.0,51.0,46.0,64.0,46.0,41.0,56.0,...,1442.0,1743.0,2308.0,2768.0,2416.0,3791.0,4366.0,7949.0,11129.0,25561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,58.0,53.0,58.0,57.0,61.0,57.0,55.0,52.0,58.0,58.0,...,151.0,167.0,182.0,218.0,248.0,313.0,342.0,348.0,822.0,2328.0
2025-04-01,59.0,54.0,59.0,58.0,62.0,58.0,56.0,53.0,59.0,59.0,...,154.0,168.0,183.0,226.0,237.0,314.0,374.0,351.0,631.0,1666.0
2025-05-01,83.0,76.0,84.0,81.0,87.0,82.0,79.0,75.0,83.0,84.0,...,217.0,238.0,256.0,319.0,346.0,422.0,523.0,543.0,521.0,1639.0


In [16]:
result = np.trace(flipped_df, offset=-1, dtype=int)
result

np.int64(8264)

In [17]:
retention_matrix_e

maturity,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
subscription_bom,,,,,,,,,,,,,,,,,,,,,
2019-12-01,1.0,0.420200,0.284300,0.202600,0.155700,0.125300,0.108900,0.072500,0.036100,0.046500,...,0.003400,0.003100,0.003500,0.003200,0.003100,0.002800,0.002000,0.003100,0.002400,0.002700
2020-01-01,1.0,0.432200,0.264700,0.206400,0.154400,0.128400,0.083500,0.078900,0.055600,0.072100,...,0.002200,0.002400,0.002200,0.001700,0.001700,0.001800,0.001900,0.001400,0.001400,0.001527
2020-02-01,1.0,0.415900,0.260300,0.210500,0.146900,0.099300,0.095700,0.060200,0.080600,0.066900,...,0.001800,0.002300,0.001700,0.001600,0.001400,0.002300,0.001800,0.001900,0.001696,0.001851
2020-03-01,1.0,0.447400,0.280400,0.180700,0.137300,0.117600,0.077200,0.094200,0.079600,0.059000,...,0.002000,0.002400,0.002200,0.001200,0.002400,0.001500,0.001500,0.001670,0.001491,0.001627
2020-04-01,1.0,0.435400,0.311000,0.170800,0.148300,0.094500,0.108300,0.090300,0.068200,0.056400,...,0.002200,0.001600,0.001800,0.002500,0.001800,0.002000,0.001887,0.001823,0.001693,0.001847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-01,1.0,0.353100,0.149500,0.146900,0.134500,0.106515,0.093582,0.078364,0.071832,0.064876,...,0.025077,0.024756,0.022460,0.023542,0.024541,0.025994,0.024334,0.025024,0.022790,0.024862
2025-04-01,1.0,0.378800,0.210700,0.224500,0.188203,0.142162,0.135673,0.109930,0.100863,0.092419,...,0.035626,0.035170,0.031908,0.033444,0.034864,0.036928,0.034570,0.035550,0.032376,0.035320
2025-05-01,1.0,0.317900,0.331300,0.319213,0.257456,0.211321,0.194857,0.156093,0.145196,0.132530,...,0.051071,0.050417,0.045741,0.047944,0.049980,0.052938,0.049557,0.050963,0.046413,0.050632


In [18]:
max_diag_length = min(flipped_df.shape)

for i in range(1, max_diag_length):
    # Extract the diagonal element with an offset of 1 from the left and print its value
    print(flipped_df.iat[i, i - 1])

19.0
21.0
21.0
48.0
61.0
44.0
45.0
37.0
64.0
60.0
55.0
53.0
153.0
107.0
77.0
47.0
32.0
21.0
24.0
46.0
38.0
47.0
37.0
45.0
96.0
58.0
46.0
28.0
28.0
16.0
21.0
24.0
37.0
80.0
114.0
77.0
199.0
150.0
103.0
59.0
74.0
45.0
63.0
90.0
100.0
111.0
150.0
128.0
208.0
173.0
95.0
75.0
117.0
100.0
102.0
148.0
172.0
154.0
316.0
331.0
534.0
322.0
248.0
314.0
523.0
579.0
654.0


In [19]:
result_df = pd.DataFrame()
result_df['subscription_bom'] = retention_matrix_e2.index

# Extract the diagonal values from the flipped_df
diagonal_values = [flipped_df.iat[i, i - 1] for i in range(1, min(flipped_df.shape))]

# The diagonal values correspond to the 'nb_boxes' for each subscription_bom,
# starting from the second row of retention_matrix_e2.
# We need to align these with the correct subscription_bom.
# The first row of result_df will have a NaN for 'nb_boxes' as there's no
# corresponding value in the flipped_df diagonal at offset -1 for the first row.
result_df['nb_boxes'] = [np.nan] + diagonal_values

display(result_df)

,subscription_bom,nb_boxes
0,2019-12-01,NaN
1,2020-01-01,19.0
2,2020-02-01,21.0
3,2020-03-01,21.0
4,2020-04-01,48.0
...,...,...
63,2025-03-01,248.0
64,2025-04-01,314.0
65,2025-05-01,523.0
66,2025-06-01,579.0


In [20]:
result_df['subscription_bom'] = pd.to_datetime(result_df['subscription_bom'])

result_df['year'] = result_df['subscription_bom'].dt.year

yearly_nb_boxes = result_df[result_df['year'] >= 2020].groupby('year')['nb_boxes'].sum().reset_index()

display(yearly_nb_boxes)

,year,nb_boxes
0,2020,528.0
1,2021,674.0
2,2022,625.0
3,2023,1272.0
4,2024,1991.0
5,2025,3174.0
